# Chicago Salary Analysis

In [1]:
# Setup: import and ensure required libraries (polars/pandas/statsmodels/etc.).
import sys, subprocess, importlib
def ensure(pkg, extra=None):
    try:
        return importlib.import_module(pkg)
    except ImportError:
        base = pkg.split('.')[0]
        pip_name = {'sklearn':'scikit-learn', 'gender_guesser':'gender-guesser'}.get(base, base)
        to_install = [pip_name] + (extra or [])
        subprocess.run([sys.executable, '-m', 'pip', 'install'] + to_install, check=True)
        return importlib.import_module(pkg)
pl = ensure('polars', ['pyarrow'])
pd = ensure('pandas')
np = ensure('numpy')
sm = ensure('statsmodels.api')
sms = ensure('statsmodels.stats.api')
sm_diag = ensure('statsmodels.stats.diagnostic')
sm_tools = ensure('statsmodels.tools.tools')
seaborn = ensure('seaborn')
gender_guesser = ensure('gender_guesser')
from patsy import dmatrices
# Use non-interactive backend to avoid macOS GUI warnings and speed up plotting
import os as _os
import matplotlib
# Only force Agg backend when not running inside IPython/Jupyter
_IS_IPY = False
try:
    from IPython import get_ipython as _get_ipy
    _IS_IPY = bool(_get_ipy())
except Exception:
    _IS_IPY = False
if not _IS_IPY:
    matplotlib.use(_os.environ.get('MPLBACKEND', 'Agg'))
import matplotlib.pyplot as plt
def show_if_interactive():
    try:
        be = matplotlib.get_backend().lower()
    except Exception:
        be = 'agg'
    if be not in ('agg','pdf','ps','svg','template'):
        try:
            plt.show()
        except Exception:
            pass
sklearn = ensure('sklearn')
shap = ensure('shap')
try:
    xgb = importlib.import_module('xgboost')
except Exception:
    xgb = None
# Optional: PyTorch for GPU (MPS on Apple Silicon) detection
try:
    torch = ensure('torch')
    HAS_MPS = hasattr(torch.backends, 'mps') and torch.backends.mps.is_available()
    HAS_CUDA = torch.cuda.is_available()
except Exception:
    try:
        torch = importlib.import_module('torch')
        HAS_MPS = hasattr(torch.backends, 'mps') and torch.backends.mps.is_available()
        HAS_CUDA = torch.cuda.is_available()
    except Exception:
        torch = None
        HAS_MPS = False
        HAS_CUDA = False
print('Ready: polars', pl.__version__)


Ready: polars 1.33.1


In [2]:
# Run configuration
SAMPLE_N = None   # set to None for full dataset
RUN_HEAVY = True  # enable heavy analyses (Oaxaca/PS/quantile) when N is sufficient
# Accuracy/speed tradeoffs for model-based parts (SHAP etc.)
HIGH_ACCURACY = True
SHAP_EVAL_N = 10000 if HIGH_ACCURACY else 5000
XGB_N_EST = 900 if HIGH_ACCURACY else 400
XGB_LR = 0.05 if HIGH_ACCURACY else 0.1


In [3]:
# Load data: read Chicago salaries CSV with Polars (full dataset; no sampling).
from pathlib import Path
DATA = Path('data/chicago_salaries.csv')
if not DATA.exists():
    raise FileNotFoundError('Expected data/chicago_salaries.csv')
try:
    df = pl.read_csv(str(DATA), try_parse_dates=False, infer_schema_length=1000)
except Exception:
    df = pl.read_csv(str(DATA), try_parse_dates=False, infer_schema_length=1000, encoding='utf8-lossy')
# Subsample for quick compile/run
# df = df.sample(n=1000, with_replacement=False, shuffle=True, seed=42)
df.shape, df.columns
# Subsample for quick compile/run (uses SAMPLE_N)
# try:
#     SAMPLE_N
# except NameError:
#     SAMPLE_N = 1000
# if SAMPLE_N:
#     df = df.sample(n=min(SAMPLE_N, df.height), with_replacement=False, shuffle=True, seed=42)
# print('Using sample size:', len(df))


FileNotFoundError: Expected data/chicago_salaries.csv

In [ ]:
# Feature engineering: normalize text, infer job_level/job_family, compute annualized pay, derive log_annual, and infer gender via gender_guesser.
import re
roman_map = {
    'I':1,'II':2,'III':3,'IV':4,'V':5,'VI':6,'VII':7,'VIII':8,'IX':9,'X':10
}
def extract_job_level(title: str) -> int:
    if title is None:
        return 0
    m = re.search(r'(?:\b| )(?:(?:I{1,3})|IV|V|VI{0,3}|VII|VIII|IX|X)(?:\b)', title)
    if m:
        return roman_map.get(m.group(0).strip(), 0)
    return 0

def simplify_job_family(title: str) -> str:
    if not title:
        return 'UNKNOWN'
    # Take part before ' (' or first comma; keep first 2 words
    base = re.split(r'\s*\(|,', title)[0]
    parts = base.strip().split()[:3]
    return ' '.join(parts) if parts else 'UNKNOWN'

def to_float(s: str) -> float:
    if s is None:
        return None
    s2 = re.sub(r'[^0-9.]+', '', str(s))
    try:
        return float(s2) if s2 else None
    except Exception:
        return None

# Normalize text columns
df = df.with_columns([
    pl.col('Name').cast(pl.Utf8).str.strip_chars().str.to_uppercase().alias('name'),
    pl.col('JobTitle').cast(pl.Utf8).str.strip_chars().str.to_uppercase().alias('job_title'),
    pl.col('Department').cast(pl.Utf8).str.strip_chars().str.to_uppercase().alias('department'),
    pl.col('Time').cast(pl.Utf8).str.strip_chars().str.to_uppercase().alias('time'),
    pl.col('PayType').cast(pl.Utf8).str.strip_chars().str.to_uppercase().alias('pay_type'),
]).drop(['Name','JobTitle','Department','Time','PayType'])



# Extract first name from 'LAST, FIRST MI' and infer gender via gender_guesser
def extract_first_name(full: str) -> str:
    if not full: return None
    parts = full.split(',')
    rhs = parts[1].strip() if len(parts) >= 2 else full
    token = rhs.split()
    return token[0].strip(" . '") if token else None

names_py = df.select(pl.col('name')).to_series().to_list()
firsts_py = [extract_first_name(n) for n in names_py]
from gender_guesser import detector as _gd
_det = _gd.Detector(case_sensitive=False)
# Save gender corpus info (package version and data file hints)
import os, importlib, sys
os.makedirs('artifacts', exist_ok=True)
try:
    pkg = importlib.import_module('gender_guesser')
    try:
        import importlib.metadata as md
        ver = md.version('gender-guesser')
    except Exception:
        ver = getattr(pkg, '__version__', 'unknown')
    pkg_path = os.path.dirname(pkg.__file__) if hasattr(pkg,'__file__') else 'unknown'
    # scan for likely dictionary files
    data_files = []
    if pkg_path != 'unknown':
        for root, dirs, files in os.walk(pkg_path):
            for f in files:
                if ('nam' in f.lower() or 'dict' in f.lower()) and f.lower().endswith('.txt'):
                    data_files.append(os.path.join(root,f))
    with open('artifacts/gender_corpus_info.txt','w') as fh:
        fh.write(f'gender_guesser version: {ver}\n')
        fh.write(f'package path: {pkg_path}\n')
        fh.write('candidate data files:\n')
        for pth in data_files[:10]:
            fh.write('  - ' + pth + '\n')
except Exception as e:
    open('artifacts/gender_corpus_info.txt','w').write('Unable to determine corpus info: ' + str(e))
raw_gender = [_det.get_gender(fn or '') for fn in firsts_py]
def map_gender(g):
    if g in ('male','mostly_male'): return 'MALE'
    if g in ('female','mostly_female'): return 'FEMALE'
    if g == 'andy': return 'ANDROGYNOUS'
    return 'UNKNOWN'
gender_cat = [map_gender(g) for g in raw_gender]
gender_bin_male = [1 if v=='MALE' else (0 if v=='FEMALE' else None) for v in gender_cat]
df = df.with_columns([
    pl.Series('first_name', firsts_py, dtype=pl.Utf8),
    pl.Series('gender_guess', gender_cat, dtype=pl.Utf8),
    pl.Series('is_male', gender_bin_male, dtype=pl.Int64)
])

# Clean numeric columns
salary_clean = pl.col('Salary').cast(pl.Utf8).str.replace_all(r"[^0-9.]", "")
rate_clean   = pl.col('Rate').cast(pl.Utf8).str.replace_all(r"[^0-9.]", "")
df = df.with_columns([
    pl.col('Hours').cast(pl.Float64, strict=False).alias('hours'),
    pl.when(salary_clean.str.len_chars() == 0).then(None).otherwise(salary_clean.cast(pl.Float64, strict=False)).alias('salary_num'),
    pl.when(rate_clean.str.len_chars() == 0).then(None).otherwise(rate_clean.cast(pl.Float64, strict=False)).alias('rate_num'),
])

# Derived features
df = df.with_columns([
    (pl.col('time') == 'F').alias('is_full_time'),
    (pl.col('pay_type') == 'SALARY').alias('is_salary'),
])

# Job level and family (via apply to avoid complex vectorized regex)
job_levels = df.select(pl.col('job_title')).to_series().to_list()
levels_py = [extract_job_level(t) for t in job_levels]
families_py = [simplify_job_family(t) for t in job_levels]
df = df.with_columns([
    pl.Series('job_level', levels_py, dtype=pl.Int64),
    pl.Series('job_family', families_py, dtype=pl.Utf8).str.to_uppercase()
])

# Annualized pay
hours_proxy = pl.when(pl.col('hours').is_not_null()).then(pl.col('hours'))\
    .when(pl.col('is_full_time')).then(pl.lit(40.0))\
    .otherwise(pl.lit(20.0))
annual_from_hourly = (pl.col('rate_num') * hours_proxy * pl.lit(52.0))
annualized = pl.when(pl.col('is_salary')).then(pl.col('salary_num'))\
    .when(pl.col('rate_num').is_not_null()).then(annual_from_hourly)\
    .otherwise(pl.col('salary_num'))
df = df.with_columns([
    annualized.alias('annualized_pay'),
    pl.when(annualized > 0).then(annualized.log()).otherwise(None).alias('log_annual')
])


df.shape


In [ ]:
# Missingness summary
missing = (df.select([pl.col(c).is_null().sum().alias(c) for c in df.columns])
             .transpose(include_header=True, header_name='column', column_names=['n_missing']))
display(missing.sort('n_missing', descending=True).head(20))
# Distribution
pdf = df.select(['annualized_pay','log_annual','department','job_family','is_full_time']).to_pandas()
fig, ax = plt.subplots(1,2, figsize=(12,4))
seaborn.histplot(data=pdf, x='annualized_pay', ax=ax[0], bins=50)
ax[0].set_title('Annualized pay (raw)')
seaborn.histplot(data=pdf, x='log_annual', ax=ax[1], bins=50)
ax[1].set_title('Annualized pay (log)')
plt.tight_layout()
import os
os.makedirs('artifacts', exist_ok=True)
fig.savefig('artifacts/pay_distribution_hist.png', dpi=200, bbox_inches='tight')
show_if_interactive()
# Top departments/job families
top_depts = (df.group_by('department').len().sort('len', descending=True).head(10))
top_fams = (df.group_by('job_family').len().sort('len', descending=True).head(10))
display(top_depts)
display(top_fams)
try:
    top_depts.write_csv('artifacts/top_departments.csv')
    top_fams.write_csv('artifacts/top_job_families.csv')
    # Save quick bar figures for top categories
    tdp = top_depts.to_pandas(); tfp = top_fams.to_pandas()
    fig2, ax2 = plt.subplots(1,2, figsize=(14,4))
    seaborn.barplot(x='len', y='department', data=tdp, ax=ax2[0])
    ax2[0].set_title('Top Departments by Count')
    seaborn.barplot(x='len', y='job_family', data=tfp, ax=ax2[1])
    ax2[1].set_title('Top Job Families by Count')
    fig2.tight_layout(); fig2.savefig('artifacts/top_categories_bars.png', dpi=200, bbox_inches='tight')
    plt.close(fig2)
except Exception:
    pass


In [ ]:
# Counts
gender_counts = df.group_by('gender_guess').len().sort('len', descending=True)
display(gender_counts)

# Pay by gender (median and IQR on annualized pay)
gb = (df.select(['gender_guess','annualized_pay']).drop_nulls().group_by('gender_guess')
      .agg([pl.col('annualized_pay').median().alias('median_pay'),
            (pl.col('annualized_pay').quantile(0.75)-pl.col('annualized_pay').quantile(0.25)).alias('IQR')])
      .sort('median_pay', descending=True))
display(gb)


In [ ]:

# Prepare data for statsmodels (dynamic hours basis with fallback)
import warnings
warnings.filterwarnings('ignore', message='covariance of constraints does not have full rank', module='statsmodels')
warnings.filterwarnings('ignore', category=RuntimeWarning, module='statsmodels')
model_df = df.select(['log_annual','department','job_family','gender_guess','is_salary','job_level','hours']).drop_nulls(subset=['log_annual','department','job_family','gender_guess','is_salary','job_level','hours']).to_pandas()
model_df['department'] = model_df['department'].astype('category')
model_df['job_family'] = model_df['job_family'].astype('category')
# Cap high-cardinality categoricals to reduce singular design matrices
import pandas as pd, numpy as np

def cap_top_n(s, n=10):
    vc = s.value_counts(dropna=False)
    keep = set(vc.head(n).index.tolist())
    return s.where(s.isin(keep), other='OTHER')
model_df['department_cap'] = cap_top_n(model_df['department'].astype(str), 10).astype('category')
model_df['job_family_cap'] = cap_top_n(model_df['job_family'].astype(str), 10).astype('category')
# Within-group centered hours (reduces collinearity with FE dummies)
model_df['hours_w'] = model_df.groupby(['department_cap','job_family_cap'], observed=True)['hours'].transform(lambda s: s - s.mean())
# Use binary male indicator to avoid redundant gender dummies
model_df['male01'] = (model_df['gender_guess'] == 'MALE').astype(int)

from patsy import dmatrices, bs
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from numpy.polynomial.legendre import legvander

# Helper to add Legendre orthogonal polynomial features for hours
def add_legendre_hours(df, col='hours_w', deg=3, prefix='hoursW_L'):
    x = pd.to_numeric(df[col], errors='coerce')
    # Map to [-1,1] for Legendre
    xmin, xmax = x.min(), x.max()
    z = 2*(x - xmin)/(xmax - xmin) - 1
    V = legvander(z.fillna(z.mean()), deg)  # includes degree 0 column (const)
    for d in range(1, deg+1):
        df[f"{prefix}{d}"] = V[:, d]
    return df

# Formulas
f_bs   = 'log_annual ~ male01 + C(department_cap) + C(job_family_cap) + job_level + bs(hours_w, df=5, include_intercept=False)'
f_leg  = 'log_annual ~ male01 + C(department_cap) + C(job_family_cap) + job_level + hoursW_L1 + hoursW_L2 + hoursW_L3'

used = 'bs'
try:
    y, X = dmatrices(f_bs, data=model_df, return_type='dataframe')
    ols = sm.OLS(y, X).fit()
    exog = np.asarray(X)
    rank = int(np.linalg.matrix_rank(exog))
    ncols = X.shape[1]
    vifs = []
    for j in range(ncols):
        try:
            vifs.append(float(variance_inflation_factor(exog, j)))
        except Exception:
            vifs.append(np.inf)
    bad = (rank < ncols) or any((not np.isfinite(v)) or (v>50) for v in vifs)
    if bad:
        print('Switching to Legendre orthogonal polynomials for hours (spline basis caused rank/VIF issues).')
        model_df = add_legendre_hours(model_df, 'hours', 3)
        y, X = dmatrices(f_leg, data=model_df, return_type='dataframe')
        ols = sm.OLS(y, X).fit()
        used = 'legendre'
except Exception as e:
    print('Spline fit failed ({}). Falling back to Legendre basis.'.format(e))
    model_df = add_legendre_hours(model_df, 'hours', 3)
    y, X = dmatrices(f_leg, data=model_df, return_type='dataframe')
    ols = sm.OLS(y, X).fit()
    used = 'legendre'

# Compact coefficient print to avoid summary-induced warnings
def print_coef(res, title):
    names = list(res.model.exog_names) if hasattr(res,'model') and hasattr(res.model,'exog_names') else []
    p = res.params
    if hasattr(p, 'index'):
        idx = p.index; coef_series = p
    else:
        idx = names if names else [f'X{i}' for i in range(len(p))]; coef_series = pd.Series(p, index=idx)
    bs_ = res.bse if hasattr(res,'bse') else None
    se_series = bs_ if hasattr(bs_, 'index') else pd.Series(bs_, index=idx) if bs_ is not None else pd.Series(np.nan, index=idx)
    tvals = coef_series / se_series
    pvals = res.pvalues if hasattr(res,'pvalues') else pd.Series(np.nan, index=idx)
    out = pd.DataFrame({'coef': coef_series, 'se': se_series, 't': tvals, 'pval': pvals})
    print(title)
    print(out.head(30).to_string())

print('Hours basis used:', used)
print_coef(ols, 'OLS (non-robust, compact)')

# Heteroskedasticity tests
from statsmodels.stats.diagnostic import het_breuschpagan, het_white
bp_lm, bp_p, fval, f_p = het_breuschpagan(ols.resid, X)
w_stat, w_p, f_w, f_wp = het_white(ols.resid, X)
print('Breusch-Pagan: LM p={:.4g}, F p={:.4g}'.format(bp_p, f_p))
print('White test: stat p={:.4g}, F p={:.4g}'.format(w_p, f_wp))

# Robust SEs (HC1)
ols_hc = ols.get_robustcov_results(cov_type='HC1')
print_coef(ols_hc, 'OLS (HC1 robust, compact)')

# Cluster-robust by department (guarded)
clusters = model_df['department'].cat.codes.values
try:
    if len(clusters)==len(ols.resid) and len(set(clusters))>1:
        ols_cl = ols.get_robustcov_results(cov_type='cluster', groups=clusters)
        print_coef(ols_cl, 'OLS (cluster-robust, compact)')
    else:
        print('Skipping cluster-robust SEs (insufficient groups or length mismatch).')
except Exception as e:
    print('Cluster-robust failed:', e)
# Durbin-Watson for residual independence
from statsmodels.stats.stattools import durbin_watson
print('Durbin-Watson:', durbin_watson(ols.resid))


In [ ]:
# Regression diagnostics: Linearity, Independence, Homoscedasticity, Normality, Multicollinearity
import numpy as np, pandas as pd
from statsmodels.stats.diagnostic import het_breuschpagan, het_white, acorr_breusch_godfrey, linear_reset, acorr_ljungbox
from statsmodels.stats.stattools import jarque_bera, durbin_watson
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Choose a fitted model to diagnose
res = None
for name in ['ols', 'm3_ols', 'm4_ols', 'm3', 'm4']:
    if name in globals():
        res = globals()[name]
        break
if res is None:
    raise RuntimeError('No fitted model found for diagnostics (expected variables like ols, m3_ols, m4_ols).')

# Extract design and residuals
exog = np.asarray(res.model.exog)
endog = np.asarray(res.model.endog).ravel()
resid = np.asarray(res.resid).ravel()
colnames = list(getattr(res.model, 'exog_names', [f'X{i}' for i in range(exog.shape[1])]))

report = {}

# 1) Linearity (RESET test)
try:
    reset = linear_reset(res, power=2, use_f=True)
    # reset is a ContrastResults with attributes fvalue, pvalue
    report['linearity_RESET_F'] = float(getattr(reset, 'fvalue', np.nan))
    report['linearity_RESET_p'] = float(getattr(reset, 'pvalue', np.nan))
except Exception as e:
    report['linearity_RESET_F'] = np.nan
    report['linearity_RESET_p'] = np.nan

# 2) Independence (autocorrelation) — Durbin-Watson and Breusch-Godfrey lags 1..4
try:
    report['independence_DW'] = float(durbin_watson(resid))
except Exception:
    report['independence_DW'] = np.nan
for L in [1,2,4]:
    try:
        lm, lmp, fval, fp = acorr_breusch_godfrey(res, nlags=L)
        report[f'independence_BG_lags{L}_p'] = float(lmp)
    except Exception:
        report[f'independence_BG_lags{L}_p'] = np.nan

# 3) Homoscedasticity — Breusch-Pagan and White
try:
    lm, lmp, fval, fp = het_breuschpagan(resid, exog)
    report['homoscedasticity_BP_p'] = float(lmp)
except Exception:
    report['homoscedasticity_BP_p'] = np.nan
try:
    stat, pval, fstat, fpval = het_white(resid, exog)
    report['homoscedasticity_White_p'] = float(pval)
except Exception:
    report['homoscedasticity_White_p'] = np.nan

# 4) Normality of residuals — Jarque-Bera
try:
    jb_stat, jb_p, skew, kurt = jarque_bera(resid)
    report['normality_JB_p'] = float(jb_p)
    report['normality_skew'] = float(skew)
    report['normality_kurtosis'] = float(kurt)
except Exception:
    report['normality_JB_p'] = np.nan

# 5) No perfect multicollinearity — rank check, condition number, VIFs
try:
    rank = int(np.linalg.matrix_rank(exog))
    report['multicollinearity_rank'] = rank
    report['multicollinearity_ncols'] = int(exog.shape[1])
    # Condition number (based on exog)
    try:
        cond = float(np.linalg.cond(exog))
    except Exception:
        # fall back to SVD-based
        s = np.linalg.svd(exog, compute_uv=False)
        cond = float(s.max() / s.min()) if np.all(s>0) else np.inf
    report['multicollinearity_condition_number'] = cond
except Exception:
    report['multicollinearity_rank'] = np.nan
    report['multicollinearity_ncols'] = exog.shape[1]
    report['multicollinearity_condition_number'] = np.nan

# Compute VIF for non-constant columns
try:
    # Identify constant-like columns (std ~ 0) and typical intercept names
    mask_nonconst = np.array([np.std(exog[:, j]) > 1e-12 and (colnames[j].lower() not in ('const','intercept')) for j in range(exog.shape[1])])
    X_vif = exog[:, mask_nonconst]
    names_vif = [colnames[j] for j in range(exog.shape[1]) if mask_nonconst[j]]
    vif_vals = []
    for j in range(X_vif.shape[1]):
        try:
            vif_vals.append(float(variance_inflation_factor(X_vif, j)))
        except Exception:
            vif_vals.append(np.nan)
    vif_tbl = pd.DataFrame({'feature': names_vif, 'VIF': vif_vals}).sort_values('VIF', ascending=False)
except Exception:
    vif_tbl = pd.DataFrame(columns=['feature','VIF'])

# Present concise results with common decision thresholds
alpha = 0.05
print('Assumption checks (alpha=0.05):')
# Linearity
p = report.get('linearity_RESET_p', np.nan)
print('- Linearity (RESET p):', p, '->', 'OK' if (np.isnan(p) or p>alpha) else 'Potential nonlinearity')
# Independence
dw = report.get('independence_DW', np.nan)
bg1 = report.get('independence_BG_lags1_p', np.nan)
print('- Independence: DW={:.3f} (≈2 ok); BG p(lag1)={}'.format(dw if np.isfinite(dw) else np.nan, bg1))
# Homoscedasticity
bp = report.get('homoscedasticity_BP_p', np.nan)
wh = report.get('homoscedasticity_White_p', np.nan)
print('- Homoscedasticity: BP p={}, White p={}'.format(bp, wh))
# Normality
jb = report.get('normality_JB_p', np.nan)
print('- Normality (JB p):', jb)
# Multicollinearity
rank = report.get('multicollinearity_rank', np.nan)
ncols = report.get('multicollinearity_ncols', np.nan)
cond = report.get('multicollinearity_condition_number', np.nan)
print('- Rank check: rank={} vs columns={} -> {}'.format(rank, ncols, 'Full rank' if rank==ncols else 'Rank deficient'))
print('- Condition number (rule of thumb: < 30-50 acceptable):', cond)
print()
print('Top VIFs (drop const):')
try:
    display(vif_tbl.head(15))
except Exception:
    print(vif_tbl.head(15).to_string(index=False))


In [ ]:
# Influence analysis: leverage and Cook's distance (fast + cleaner)
import numpy as np, pandas as pd, matplotlib.pyplot as plt, os
import statsmodels.api as sm

# Select model for influence (prefer FE OLS m3 if available)
res_inf = None
for name in ['m3', 'ols']:
    if name in globals():
        res_inf = globals()[name]
        break
if res_inf is None:
    raise RuntimeError('No fitted model found for influence analysis (expected m3 or ols).')

# Guard against heavy influence computations on large X
n = int(len(res_inf.model.endog))
try:
    p = int(res_inf.model.exog.shape[1])
except Exception:
    p = 0
max_work = 8_000_000  # threshold for n*p; above this we subsample

if n * max(1,p) > max_work:
    rng = np.random.default_rng(42)
    take = min(8000, n)
    idx = rng.choice(n, size=take, replace=False)
    res_sub = sm.OLS(res_inf.model.endog[idx], res_inf.model.exog[idx, :]).fit()
    infl = res_sub.get_influence()
    hat = np.asarray(infl.hat_matrix_diag)
    try:
        stud = np.asarray(infl.resid_studentized_internal)
    except Exception:
        stud = np.asarray(res_sub.resid)
    cooks = np.asarray(infl.cooks_distance[0])
    obs = idx
else:
    infl = res_inf.get_influence()
    # Avoid summary_frame() to skip expensive LOO computations
    hat = np.asarray(infl.hat_matrix_diag)
    try:
        stud = np.asarray(infl.resid_studentized_internal)
    except Exception:
        stud = np.asarray(res_inf.resid)
    cooks = np.asarray(infl.cooks_distance[0])
    obs = np.arange(n)

os.makedirs('artifacts', exist_ok=True)
# Save top-20 by Cook's D
(pd.DataFrame({'obs': obs, 'hat_diag': hat, 'student_resid': stud, 'cooks_d': cooks})
   .sort_values('cooks_d', ascending=False)
   .head(20)
   .to_csv('artifacts/influence_top20.csv', index=False))

# Bubble scatter (size=Cooks D), annotate top-10 only
s = 2000 * np.clip(cooks, 0, np.nanpercentile(cooks, 95))
fig, ax = plt.subplots(1,1, figsize=(7,5))
ax.scatter(hat, stud, s=s, alpha=0.25, edgecolor='k', linewidth=0.2)
ax.axhline(0, color='gray', lw=0.8)
ax.set_xlabel('Leverage (hat)'); ax.set_ylabel('Studentized Residuals')
ax.set_title('Influence: Leverage vs Studentized Residuals (size=Cooks D)')
lab_idx = np.argsort(-cooks)[:10]
for i in lab_idx:
    ax.annotate(int(obs[i]), (hat[i], stud[i]), textcoords='offset points', xytext=(5,5), fontsize=8, color='darkred')
fig.tight_layout(); fig.savefig('artifacts/influence_scatter_clean.png', dpi=200, bbox_inches='tight'); plt.close(fig)


In [ ]:
# Residual diagnostics
fitted = ols.fittedvalues
resid = ols.resid
fig, ax = plt.subplots(1,2, figsize=(12,4))
seaborn.scatterplot(x=fitted, y=resid, ax=ax[0], s=10)
ax[0].axhline(0, color='red', lw=1); ax[0].set_title('Residuals vs Fitted')
import scipy.stats as stats
stats.probplot(resid, dist='norm', plot=ax[1])
ax[1].set_title('Normal Q-Q')
plt.tight_layout(); show_if_interactive()

# VIF for numeric regressors (updated to reflect current columns)
import numpy as np, pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

num_candidates = []
for col in ['is_salary','job_level','hours_w','hoursW_L1','hoursW_L2','hoursW_L3']:
    if col in model_df.columns:
        num_candidates.append(col)
if not num_candidates:
    print('Skipping VIF: no numeric candidate columns found in model_df.')
else:
    X_num = model_df[num_candidates].apply(pd.to_numeric, errors='coerce').dropna()
    # drop constant columns
    keep = [c for c in X_num.columns if X_num[c].std(ddof=0) > 0]
    X_num = X_num[keep]
    if len(keep) >= 2 and len(X_num) >= 3:
        Xn = add_constant(X_num, has_constant='add')
        vifs = []
        for j, col in enumerate(Xn.columns):
            if col == 'const':
                continue
            try:
                v = variance_inflation_factor(Xn.values, j)
                vifs.append({'feature': col, 'VIF': float(v) if np.isfinite(v) else np.nan})
            except Exception:
                vifs.append({'feature': col, 'VIF': np.nan})
        vif_df = pd.DataFrame(vifs).sort_values('VIF', ascending=False)
        display(vif_df)
    else:
        print('Skipping VIF: not enough variable features or rows.')


In [ ]:
import numpy as np
hc_table = ols_hc.summary2().tables[1]
hc_table = hc_table.rename(columns={'Coef.':'coef','P>|t|':'pval'})
hc_table['effect_pct'] = (np.exp(hc_table['coef']) - 1.0) * 100.0
# Show top 10 by absolute effect (excluding intercept)
hc_table_f = hc_table.loc[hc_table.index != 'Intercept'].copy()
hc_table_f['abs_effect'] = hc_table_f['effect_pct'].abs()
display(hc_table_f.sort_values('abs_effect', ascending=False).head(10)[['coef','effect_pct','pval']])


In [ ]:
# SHAP with plots (guarded, MPS-accelerated path when available)
import numpy as np
import shap
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

base = df.select(['log_annual','department','job_family','gender_guess','is_full_time','is_salary','job_level','hours']).drop_nulls(subset=['log_annual']).to_pandas().copy()
# reduce cardinality
def cap_top_n(s, n=20):
    vc = s.value_counts(dropna=False)
    keep = set(vc.head(n).index.tolist())
    return s.where(s.isin(keep), other='OTHER')
base['department_cap'] = cap_top_n(base['department'].astype(str), 20)
base['job_family_cap'] = cap_top_n(base['job_family'].astype(str), 20)
cat_cols = ['department_cap','job_family_cap','gender_guess']
num_cols = ['is_full_time','is_salary','job_level','hours']
y = base['log_annual'].values
X = base[cat_cols+num_cols]
if len(base) < 30:
    print('Skipping SHAP: too few rows after filtering (', len(base), ').')
else:
    pre = ColumnTransformer([
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols),
        ('num', 'passthrough', num_cols)
    ])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    USE_MPS_SHAP = bool(globals().get('HAS_MPS', False) and os.environ.get('USE_MPS_SHAP', '1') != '0')
    model = None
    shap_values = None
    feature_names = None
    X_full = None
    if USE_MPS_SHAP:
        try:
            import torch
            import torch.nn as nn
            import torch.optim as optim
            DEVICE = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
            Xtr = pre.fit_transform(X_train).astype(np.float32)
            Xte = pre.transform(X_test).astype(np.float32)
            X_full = pre.transform(X).astype(np.float32)
            feature_names = pre.get_feature_names_out().tolist()
            ytr = y_train.astype(np.float32)
            yte = y_test.astype(np.float32)
            class MLP(nn.Module):
                def __init__(self, in_dim):
                    super().__init__()
                    self.net = nn.Sequential(
                        nn.Linear(in_dim, 256), nn.ReLU(),
                        nn.Linear(256, 128), nn.ReLU(),
                        nn.Linear(128, 1)
                    )
                def forward(self, x):
                    return self.net(x)
            mlp = MLP(Xtr.shape[1]).to(DEVICE)
            opt = optim.Adam(mlp.parameters(), lr=1e-3)
            lossf = nn.MSELoss()
            xb = torch.from_numpy(Xtr).to(DEVICE)
            yb = torch.from_numpy(ytr).view(-1,1).to(DEVICE)
            mlp.train()
            for _ in range(80):
                opt.zero_grad(); pred = mlp(xb); loss = lossf(pred, yb); loss.backward(); opt.step()
            mlp.eval()
            with torch.no_grad():
                r2_tr = 1.0 - float(((mlp(xb)-yb)**2).mean().cpu().item())/float(np.var(ytr)) if np.var(ytr)>0 else float('nan')
                r2_te = 1.0 - float(((mlp(torch.from_numpy(Xte).to(DEVICE))-torch.from_numpy(yte).view(-1,1).to(DEVICE))**2).mean().cpu().item())/float(np.var(yte)) if np.var(yte)>0 else float('nan')
            print('SHAP MLP (MPS) R2 train:', r2_tr, 'val:', r2_te)
            # SHAP DeepExplainer for PyTorch
            rng = np.random.RandomState(42)
            bg_idx = rng.choice(len(X_full), size=min(200, len(X_full)), replace=False)
            background = torch.from_numpy(X_full[bg_idx]).to(DEVICE)
            eval_idx = rng.choice(len(X_full), size=min(1500, len(X_full)), replace=False)
            X_eval = torch.from_numpy(X_full[eval_idx]).to(DEVICE)
            explainer = shap.DeepExplainer(mlp, background)
            sv = explainer.shap_values(X_eval)
            sv_np = sv[0].detach().cpu().numpy() if isinstance(sv, (list, tuple)) else sv.detach().cpu().numpy()
            shap_values = sv_np
            X_full = X_full[eval_idx]
            SHAP_INFO = {'model': 'mlp_mps', 'n_eval': int(X_full.shape[0])}
        except Exception as e:
            print('SHAP MLP (MPS) failed:', e, '— falling back to CPU tree model.')
            USE_MPS_SHAP = False
    if not USE_MPS_SHAP:
        try:
            import xgboost as _xgb
            params = dict(n_estimators=globals().get('XGB_N_EST', 400), max_depth=8,
                          learning_rate=globals().get('XGB_LR', 0.1), subsample=0.8,
                          colsample_bytree=0.8, reg_lambda=1.0, min_child_weight=1.0,
                          tree_method='hist', n_jobs=-1, random_state=42)
            if 'HAS_CUDA' in globals() and HAS_CUDA:
                params['tree_method'] = 'gpu_hist'
            model = _xgb.XGBRegressor(**params)
            shap_model_name = 'xgb_gpu' if params.get('tree_method')=='gpu_hist' else 'xgb_cpu'
        except Exception:
            model = RandomForestRegressor(n_estimators=300 if globals().get('HIGH_ACCURACY', False) else 200, max_depth=10 if globals().get('HIGH_ACCURACY', False) else 8, random_state=42, n_jobs=-1)
            shap_model_name = 'rf_cpu'
        pipe = Pipeline(steps=[('pre', pre), ('model', model)])
        pipe.fit(X_train, y_train)
        pred = pipe.predict(X_test)
        print('R2 train:', r2_score(y_train, pipe.predict(X_train)), 'val R2:', r2_score(y_test, pred))
        pre_fitted = pipe.named_steps['pre']
        feature_names = pre_fitted.get_feature_names_out().tolist()
        X_full = pre_fitted.transform(X)
        import numpy as _np
        X_full = _np.asarray(X_full, dtype=float)
        rng = np.random.RandomState(42)
        eval_idx = rng.choice(X_full.shape[0], size=min(int(globals().get('SHAP_EVAL_N', 5000)), X_full.shape[0]), replace=False)
        bg_idx = rng.choice(X_full.shape[0], size=min(1000, X_full.shape[0]), replace=False)
        X_eval = X_full[eval_idx]
        X_bg = X_full[bg_idx]
        try:
            explainer = shap.TreeExplainer(pipe.named_steps['model'], data=X_bg, feature_perturbation='interventional')
            shap_values = explainer.shap_values(X_eval)
        except Exception:
            explainer = shap.Explainer(pipe.named_steps['model'], X_bg)
            sv = explainer(X_eval)
            shap_values = sv.values if hasattr(sv, 'values') else sv
        X_full = X_eval
        SHAP_INFO = {'model': shap_model_name, 'n_eval': int(X_full.shape[0])}

    # SHAP plots (beeswarm and bar)
    shap.summary_plot(shap_values, X_full, feature_names=feature_names, show=False)
    os.makedirs('artifacts', exist_ok=True)
    plt.savefig('artifacts/shap_summary_beeswarm.png', dpi=200, bbox_inches='tight')
    plt.title('SHAP Summary (beeswarm): contribution to log(annualized pay)')
    show_if_interactive()
    shap.summary_plot(shap_values, X_full, feature_names=feature_names, plot_type='bar', show=False)
    plt.savefig('artifacts/shap_importance_bar.png', dpi=200, bbox_inches='tight')
    plt.title('Mean |SHAP| values (global importance)')
    show_if_interactive()
    # Dependence plots for top 3 features
    mean_abs = np.abs(shap_values).mean(axis=0)
    top_idx = np.argsort(-mean_abs)[:3]
    for idx in top_idx:
        feat = feature_names[idx]
        shap.dependence_plot(feat, shap_values, X_full, feature_names=feature_names, show=False)
        os.makedirs('artifacts', exist_ok=True)
        safe = ''.join(c if c.isalnum() or c in '._-' else '_' for c in feat)[:80]
        plt.savefig(f'artifacts/shap_dependence_{safe}.png', dpi=200, bbox_inches='tight')
        plt.title(f'SHAP dependence: {feat}')
        show_if_interactive()


In [ ]:
# Summarize SHAP importances if available
try:
    import numpy as np, pandas as pd
    mean_abs = np.abs(shap_values).mean(axis=0)
    mean_signed = shap_values.mean(axis=0)
    imp_tbl = pd.DataFrame({'feature': feature_names, 'mean_abs_shap': mean_abs, 'mean_signed_shap': mean_signed}).sort_values('mean_abs_shap', ascending=False)
    display(imp_tbl.head(20))
    print('Top 10 features:')
    for _, row in imp_tbl.head(10).iterrows():
        trend = 'increase' if row['mean_signed_shap'] > 0 else 'decrease'
        print('- {}: mean SHAP={:.4f}, tends to {} log pay'.format(row['feature'], row['mean_signed_shap'], trend))
except Exception as e:
    print('SHAP summary unavailable:', e)


In [ ]:
import pandas as pd, numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression
# Focus on observations with MALE or FEMALE only
gdf = df.filter(pl.col('gender_guess').is_in(['MALE','FEMALE']))
gdf = gdf.drop_nulls(subset=['log_annual'])
gpd = gdf.select(['log_annual','annualized_pay','gender_guess','hours','job_level','is_full_time','is_salary','department','job_family']).to_pandas()
gpd['male01'] = (gpd['gender_guess'] == 'MALE').astype(int)
# Basic shapes
print('N (male,female)=', gpd['male01'].sum(), (1-gpd['male01']).sum())
print('Overall N=', len(gpd))
# Descriptives by gender
desc = gpd.groupby('male01').agg({
    'log_annual':['mean','std'],
    'hours':['mean','median'],
    'job_level':['mean','median'],
    'is_full_time':['mean'],
    'is_salary':['mean'],
})
display(desc)
# Top departments by share within each gender
for label,val in [('Male',1),('Female',0)]:
    t = (gpd[gpd['male01']==val]['department'].value_counts(normalize=True).head(10)*100).round(1)
    print(f"Top departments for {label} (%):\n", t.to_string())


In [ ]:
# Standardized mean differences (SMD) for select covariates
def smd_numeric(x, g):
    x1 = x[g==1].astype(float); x0 = x[g==0].astype(float)
    m1, m0 = x1.mean(), x0.mean()
    v1, v0 = x1.var(ddof=1), x0.var(ddof=1)
    sp = np.sqrt((v1+v0)/2.0)
    return (m1-m0)/sp if sp>0 else 0.0

def smd_binary(x, g):
    p1 = x[g==1].mean(); p0 = x[g==0].mean()
    p = 0.5*(p1+p0)
    sp = np.sqrt(p*(1-p))
    return (p1-p0)/sp if sp>0 else 0.0

smds = {
    'hours': smd_numeric(gpd['hours'].fillna(gpd['hours'].median()), gpd['male01']),
    'job_level': smd_numeric(gpd['job_level'], gpd['male01']),
    'is_full_time': smd_binary(gpd['is_full_time'], gpd['male01']),
    'is_salary': smd_binary(gpd['is_salary'], gpd['male01']),
}
print('Standardized mean differences (male - female):')
for k,v in smds.items():
    print(f'  {k}: {v:.3f}')


In [ ]:
# Gender-gap models: robust for small samples and naming
import numpy as np, pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

def effect_pct_from_model(res, name='male01'):
    # Robustly find the coefficient even if patsy renames it
    names = []
    if hasattr(res, 'model') and hasattr(res.model, 'exog_names'):
        names = list(res.model.exog_names)
    elif hasattr(res, 'params'):
        p = res.params
        names = list(p.index) if hasattr(p, 'index') else []
    cand = None
    for k in names:
        if k == name or name+'[T.1]' == k or (name in str(k)):
            cand = k; break
    if cand is None:
        return np.nan, np.nan, np.nan
    # Pull est/SE, handling ndarray or Series
    p = res.params
    if hasattr(p, 'index'):
        b = p[cand]
        se = res.bse[cand] if hasattr(res, 'bse') and hasattr(res.bse, 'index') and cand in res.bse.index else np.nan
    else:
        try:
            idx = names.index(cand)
            b = p[idx]
            se = res.bse[idx] if hasattr(res, 'bse') else np.nan
        except Exception:
            return np.nan, np.nan, np.nan
    if not np.isfinite(se):
        return np.nan, np.nan, np.nan
    lo, hi = b - 1.96*se, b + 1.96*se
    to_pct = lambda x: (float(np.exp(x)) - 1.0) * 100.0
    return to_pct(b), to_pct(lo), to_pct(hi)

# Build gpd (already created earlier); filter to MALE/FEMALE only
work = gpd.copy()
vc = work['male01'].value_counts(dropna=False)
if vc.size < 2 or min(vc.values) < 3:
    print('Skipping OLS male effect: insufficient variation in male01 (N by gender = {})'.format(vc.to_dict()))
    # Welch fallback on log_annual
    g0 = work[work['male01']==0]['log_annual']; g1 = work[work['male01']==1]['log_annual']
    if len(g0) >= 2 and len(g1) >= 2:
        diff = g1.mean() - g0.mean()
        pct = (np.exp(diff)-1)*100
        print('Welch fallback (M1): male effect ≈ {:.2f}% (no CI)'.format(pct))
    else:
        print('Welch fallback unavailable: groups too small.')
else:
    # Use full sample (no per-group downsampling)
    # Fit OLS models with HC1 robust inference
    m1 = smf.ols('log_annual ~ male01', data=work).fit()
    m2 = smf.ols('log_annual ~ male01 + hours + I(hours**2) + job_level + is_salary', data=work).fit()
    m3 = smf.ols('log_annual ~ male01 + hours + I(hours**2) + job_level + C(department) + C(job_family)', data=work).fit()
    m4 = smf.ols('log_annual ~ male01*(hours + job_level) + C(department) + C(job_family)', data=work).fit()
    for lab, res in [('M1 Unadj', m1), ('M2 +Hours+Job', m2), ('M3 +FE', m3), ('M4 +Inter', m4)]:
        try:
            hc = res.get_robustcov_results(cov_type='HC1')
        except Exception:
            hc = res
        e, lo, hi = effect_pct_from_model(hc, 'male01')
        if np.isfinite(e):
            print('{} male effect: {:.2f}% (95% CI {:.2f}% to {:.2f}%)'.format(lab, e, lo, hi))
        else:
            if lab=='M1 Unadj':
                g0 = work[work['male01']==0]['log_annual']; g1 = work[work['male01']==1]['log_annual']
                if len(g0)>=2 and len(g1)>=2:
                    diff = g1.mean() - g0.mean()
                    pct = (np.exp(diff)-1)*100
                    print('M1 Welch fallback: male effect ≈ {:.2f}% (no CI)'.format(pct))
                else:
                    print('M1 Welch fallback unavailable: groups too small.')
            else:
                print('{} male effect: unavailable (dropped or singular)'.format(lab))
    # Additional nonlinear spec (hours^2)
    m5 = smf.ols('log_annual ~ male01 + hours + I(hours**2) + job_level + C(department) + C(job_family)', data=work).fit()
    for lab, res in [('M5 +FE +hours^2', m5)]:
        try:
            hc = res.get_robustcov_results(cov_type='HC1')
        except Exception:
            hc = res
        e, lo, hi = effect_pct_from_model(hc, 'male01')
        if np.isfinite(e):
            print('{} male effect: {:.2f}% (95% CI {:.2f}% to {:.2f}%)'.format(lab, e, lo, hi))
        else:
            print('{} male effect: unavailable (dropped or singular)'.format(lab))


In [ ]:
# Oaxaca-Blinder decomposition (fallback implemented if statsmodels lacks Oaxaca)
import numpy as np, pandas as pd

# Choose covariates (numeric/binary only to avoid collinearity issues)
feats = ['hours','job_level','is_full_time','is_salary']
ox = gpd[['log_annual','male01'] + feats].copy()
ox = ox.dropna(subset=['log_annual','male01'] + feats)
ox['male01'] = ox['male01'].astype(int)

try:
    from statsmodels.stats.oaxaca import Oaxaca
    exog = ox[feats]
    oax = Oaxaca(ox['log_annual'], exog, ox['male01']).fit()
    print(oax.summary())
except Exception as e:
    # Manual twofold Oaxaca-Blinder (female as reference)
    from statsmodels.api import OLS, add_constant
    d0 = ox[ox['male01']==0]
    d1 = ox[ox['male01']==1]
    if len(d0) < 10 or len(d1) < 10:
        print('Oaxaca fallback: groups too small (N0={}, N1={})'.format(len(d0), len(d1)))
    else:
        X0 = add_constant(d0[feats].astype(float))
        y0 = d0['log_annual'].astype(float)
        X1 = add_constant(d1[feats].astype(float))
        y1 = d1['log_annual'].astype(float)
        b0 = OLS(y0, X0).fit().params
        b1 = OLS(y1, X1).fit().params
        # Group mean X (include const)
        xbar0 = X0.mean(axis=0)
        xbar1 = X1.mean(axis=0)
        # Twofold (reference: female, group 0)
        explained = float(((xbar1 - xbar0) * b0).sum())
        unexplained = float((xbar1 @ b1 - xbar0 @ b0) - explained)
        # Per-feature contributions
        contrib_expl = ((xbar1 - xbar0) * b0).to_frame('explained')
        # Align indices and fill missing
        diff_b = (b1 - b0).reindex(contrib_expl.index).fillna(0.0)
        xbar1_aligned = xbar1.reindex(contrib_expl.index).fillna(0.0)
        contrib_unexpl = (xbar1_aligned * diff_b).to_frame('unexplained')
        out = pd.concat([contrib_expl, contrib_unexpl], axis=1)
        out.loc['TOTAL'] = [out['explained'].sum(), out['unexplained'].sum()]
        print('Oaxaca-Blinder (manual twofold, ref=female):')
        print('  Explained (endowments): {:.4f} (≈{:.2f}%)'.format(explained, (np.exp(explained)-1)*100))
        print('  Unexplained (coefficients): {:.4f} (≈{:.2f}%)'.format(unexplained, (np.exp(unexplained)-1)*100))
        try:
            display(out)
        except Exception:
            print(out.to_string())


In [ ]:
# Propensity score weighting (ATE)
# Always attempt weighting; use robust fallbacks for small samples or separation.
work = gpd.copy()
# Reduce cardinality for high-cardinality categoricals to stabilize logits
for col in ['department','job_family']:
    vc = work[col].astype(str).value_counts(dropna=False)
    keep = set(vc.head(20).index)
    work[col] = work[col].astype(str).where(work[col].astype(str).isin(keep), other='OTHER')

from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

# Design matrix for propensity
base_cols = ['hours','job_level','is_salary','department','job_family']
Xps = pd.get_dummies(work[base_cols], drop_first=True)
Xps = Xps.apply(pd.to_numeric, errors='coerce')
Xps = pd.DataFrame(SimpleImputer(strategy='most_frequent').fit_transform(Xps), columns=Xps.columns)
yps = work['male01'].astype(int).values

# Try to fit logistic; fallbacks for small N or separation
lr = None
err = None
for solver in ['lbfgs','liblinear','saga']:
    try:
        lr_try = LogisticRegression(max_iter=5000, solver=solver)
        lr_try.fit(Xps, yps)
        lr = lr_try
        break
    except Exception as e:
        err = e

if lr is None:
    # Fallback to numeric-only covariates
    Xnum = work[['hours','job_level','is_salary']].copy()
    Xnum = Xnum.apply(pd.to_numeric, errors='coerce')
    Xnum = pd.DataFrame(SimpleImputer(strategy='median').fit_transform(Xnum), columns=Xnum.columns)
    for solver in ['lbfgs','liblinear']:
        try:
            lr_try = LogisticRegression(max_iter=5000, solver=solver)
            lr_try.fit(Xnum, yps)
            lr = lr_try
            Xps = Xnum
            break
        except Exception as e:
            err = e

if lr is None:
    print('Propensity fit failed ({}). Using uniform weights; re-run with larger sample if possible.'.format(err))
    p = np.full_like(yps, fill_value=yps.mean(), dtype=float)
else:
    p = lr.predict_proba(Xps)[:,1]

# Clip probabilities to avoid extreme weights
p = np.clip(p, 0.01, 0.99)
# ATE weights (unstabilized)
w = np.where(yps==1, 1.0/p, 1.0/(1.0-p))
work['w'] = w

# Weighted outcome model (HC1 robust)
try:
    wres = smf.wls('log_annual ~ male01 + hours + I(hours**2) + job_level + C(department) + C(job_family)', data=work, weights=work['w']).fit(cov_type='HC1')
    e,lo,hi = (np.exp(wres.params['male01'])-1)*100, (np.exp(wres.conf_int().loc['male01',0])-1)*100, (np.exp(wres.conf_int().loc['male01',1])-1)*100
    print('Weighted (ATE) male effect: {:.2f}% (95% CI {:.2f}% to {:.2f}%)'.format(e,lo,hi))
except Exception as e:
    print('Weighted outcome model failed:', e)


In [ ]:
# Quantile regression (skip on small sample to avoid instability)
if len(gpd) < 200:
    print('Skipping quantile regression: sample too small (N={}).'.format(len(gpd)))
else:
    from statsmodels.regression.quantile_regression import QuantReg
    qr_df = gpd.copy()
    # Reduce cardinality for high-cardinality categoricals and ensure strings
    for col in ['department','job_family']:
        vc = qr_df[col].astype(str).value_counts(dropna=False)
        keep = set(vc.head(20).index)
        qr_df[col] = qr_df[col].astype(str).where(qr_df[col].astype(str).isin(keep), other='OTHER')
    # Build design matrix
    X_raw = qr_df[['male01','hours','job_level','is_salary','department','job_family']].copy()
    X_raw['hours2'] = (pd.to_numeric(X_raw['hours'], errors='coerce')**2)
    X = pd.get_dummies(X_raw, drop_first=True)
    X = X.apply(pd.to_numeric, errors='coerce')
    y = pd.to_numeric(qr_df['log_annual'], errors='coerce')
    # Drop any rows with NA after coercion and cast to float
    mask = y.notna() & X.notna().all(axis=1)
    X = X.loc[mask].astype(float)
    y = y.loc[mask].astype(float)
    X = sm.add_constant(X)
    for tau in [0.25, 0.5, 0.75]:
        res = QuantReg(y, X).fit(q=tau, max_iter=5000)
        coef = res.params.get('male01', np.nan)
        print('Quantile {:.2f}: male effect = {:.2f}%'.format(tau, (np.exp(coef)-1)*100))


In [ ]:
# Balance (ASMD): compute standardized mean differences pre- and post-weight; tabulate results.
import numpy as np, pandas as pd
# Recompute pre-PS SMDs (unweighted)
def smd_numeric(x, g):
    x1 = x[g==1].astype(float); x0 = x[g==0].astype(float)
    m1, m0 = x1.mean(), x0.mean()
    v1, v0 = x1.var(ddof=1), x0.var(ddof=1)
    sp = np.sqrt((v1+v0)/2.0)
    return float(abs(m1-m0)/sp) if sp>0 else 0.0
def smd_binary(x, g):
    p1 = x[g==1].mean(); p0 = x[g==0].mean()
    p = 0.5*(p1+p0)
    sp = np.sqrt(p*(1-p))
    return float(abs(p1-p0)/sp) if sp>0 else 0.0
# Weighted SMDs (ATE weights from 'work' DataFrame built earlier)
def wmean(x, w): return float((w*x).sum() / w.sum())
def wvar(x, w):
    mu = wmean(x, w)
    return float(((w*(x-mu)**2).sum() / w.sum()))
def w_smd_numeric(x, g, w):
    w1 = w[g==1]; x1 = x[g==1].astype(float)
    w0 = w[g==0]; x0 = x[g==0].astype(float)
    m1, m0 = wmean(x1, w1), wmean(x0, w0)
    v1, v0 = wvar(x1, w1), wvar(x0, w0)
    sp = np.sqrt((v1+v0)/2.0)
    return float(abs(m1-m0)/sp) if sp>0 else 0.0
def w_smd_binary(x, g, w):
    p1 = wmean(x[g==1], w[g==1]); p0 = wmean(x[g==0], w[g==0])
    p = 0.5*(p1+p0); sp = np.sqrt(p*(1-p))
    return float(abs(p1-p0)/sp) if sp>0 else 0.0
# Pre (gpd) and post (work) datasets
pre = gpd.copy()
post = work.copy() if 'work' in globals() else None
has_weights = (post is not None) and ('w' in post.columns)
# Compute ASMD pre and post
covs = ['hours','job_level','is_full_time','is_salary']
rows = []
for c in covs:
    if c in ['hours','job_level']:
        pre_s = smd_numeric(pre[c].astype(float), pre['male01'])
    else:
        pre_s = smd_binary(pre[c].astype(float), pre['male01'])
    if has_weights:
        if c in ['hours','job_level']:
            post_s = w_smd_numeric(post[c].astype(float), post['male01'], post['w'])
        else:
            post_s = w_smd_binary(post[c].astype(float), post['male01'], post['w'])
    else:
        post_s = np.nan
    rows.append({'covariate': c, 'asmd_pre': pre_s, 'asmd_post_weighted': post_s})
asmd_tbl = pd.DataFrame(rows).sort_values('asmd_post_weighted')
if not has_weights:
    print('Skipping post-weight ASMD: weights not available (propensity skipped).')
display(asmd_tbl)


In [ ]:
# Model comparison: aggregate effects across models (unadj/adj/FE/weighted/quantile); display and save to artifacts.
import numpy as np, pandas as pd
def eff_ci(res, name='male01'):
    try:
        est = res.params.get(name, np.nan)
        ci = res.conf_int().loc[name]
        lo, hi = ci[0], ci[1]
    except Exception:
        est, lo, hi = np.nan, np.nan, np.nan
    to_pct = lambda b: (np.exp(b)-1)*100.0
    return to_pct(est), to_pct(lo), to_pct(hi)

rows=[]
# Choose available model set
models_list = None
if 'm1_ols' in globals():
    models_list = [('M1 Unadjusted', m1_ols), ('M2 +Hours+Job', m2_ols), ('M3 +Dept+Family FE', m3_ols), ('M4 +Interactions', m4_ols)]
elif 'm1' in globals():
    models_list = [('M1 Unadjusted', m1), ('M2 +Hours+Job', m2), ('M3 +Dept+Family FE', m3), ('M4 +Interactions', m4)]
else:
    models_list = []

# Prefer robust extractor if available
use_effect = effect_pct_from_model if 'effect_pct_from_model' in globals() else eff_ci
for label,mod in models_list:
    e,lo,hi = use_effect(mod, 'male01')
    rows.append({'model':label, 'effect_pct': e, 'ci_lo': lo, 'ci_hi': hi})

# Weighted FE (wres) if available
try:
    e = (np.exp(wres.params['male01'])-1)*100
    lo = (np.exp(wres.conf_int().loc['male01',0])-1)*100
    hi = (np.exp(wres.conf_int().loc['male01',1])-1)*100
    rows.append({'model':'WLS (ATE-weighted) +FE', 'effect_pct': e, 'ci_lo': lo, 'ci_hi': hi})
except Exception:
    pass
# Quantile tau=0.5 if large enough
if len(gpd) >= 200:
    from statsmodels.regression.quantile_regression import QuantReg
    qr_df = gpd.copy()
    # Reduce cardinality and coerce to strings for dummies
    for col in ['department','job_family']:
        vc = qr_df[col].astype(str).value_counts(dropna=False)
        keep = set(vc.head(20).index)
        qr_df[col] = qr_df[col].astype(str).where(qr_df[col].astype(str).isin(keep), other='OTHER')
    X_raw = qr_df[['male01','hours','job_level','is_full_time','department','job_family']].copy()
    X = pd.get_dummies(X_raw, drop_first=True)
    X = X.apply(pd.to_numeric, errors='coerce')
    y = pd.to_numeric(qr_df['log_annual'], errors='coerce')
    mask = y.notna() & X.notna().all(axis=1)
    X = sm.add_constant(X.loc[mask].astype(float))
    y = y.loc[mask].astype(float)
    try:
        qr = QuantReg(y, X).fit(q=0.5, max_iter=5000)
        coef = qr.params.get('male01', np.nan)
        rows.append({'model':'Quantile (tau=0.5) +FE', 'effect_pct': (np.exp(coef)-1)*100, 'ci_lo': np.nan, 'ci_hi': np.nan})
    except Exception:
        pass
cmp = pd.DataFrame(rows)
display(cmp)
# Save model comparison
import os
os.makedirs('artifacts', exist_ok=True)
cmp.to_csv('artifacts/model_comparison.csv', index=False)
try:
    from tabulate import tabulate
    open('artifacts/model_comparison.md','w').write(tabulate(cmp, headers='keys', tablefmt='github', showindex=False))
except Exception:
    open('artifacts/model_comparison.md','w').write(cmp.to_string(index=False))


In [ ]:
# Robustness: Ridge regression (numeric/categorical pipeline)
import numpy as np, pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import Pipeline

# Build male01 from gender_guess (df does not have male01 column)
base = df.select(['log_annual','gender_guess','department','job_family','job_level','hours','is_salary']).drop_nulls().to_pandas()
base['male01'] = (base['gender_guess'].astype(str) == 'MALE').astype(int)
X = base[['male01','department','job_family','job_level','hours','is_salary']]
y = base['log_annual'].values
pre = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), ['department','job_family']),
    ('num', StandardScaler(), ['male01','job_level','hours','is_salary'])
])
ridge = RidgeCV(alphas=[0.1,1.0,10.0], cv=5)
pipe = Pipeline(steps=[('pre', pre), ('model', ridge)])
pipe.fit(X, y)
print('Ridge alpha selected:', pipe.named_steps['model'].alpha_)
# Counterfactual prediction at mean profile: male01=1 vs 0
X_mean = pd.DataFrame({
    'male01':[0,1],
    'department':[X['department'].mode()[0]]*2,
    'job_family':[X['job_family'].mode()[0]]*2,
    'job_level':[X['job_level'].mean()]*2,
    'hours':[X['hours'].mean()]*2,
    'is_salary':[X['is_salary'].mode()[0]]*2,
})
pred = pipe.predict(X_mean)
print('Ridge-implied male effect (at means): {:.2f}%'.format((np.exp(pred[1]-pred[0])-1)*100))


In [ ]:
# GLM: Gamma family with log link (robust to heteroskedasticity on positive outcome)
import numpy as np, pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import bs
from statsmodels.genmod.families import Gamma
from statsmodels.genmod.families.links import Log

glm_df = gpd.copy()  # uses MALE/FEMALE only subset with log_annual available
# Use level outcome with log link; reduce cardinality and avoid spline collinearity
try:
    glm_df = glm_df.dropna(subset=['annualized_pay','hours','job_level','is_salary','department','job_family']).copy()
    # Reduce high-cardinality categoricals to top N to avoid singularities
    for col in ['department','job_family']:
        vc = glm_df[col].astype(str).value_counts(dropna=False)
        keep = set(vc.head(20).index)
        glm_df[col] = glm_df[col].astype(str).where(glm_df[col].astype(str).isin(keep), other='OTHER')
    # Simpler polynomial hours terms for numerical stability
    glmr = smf.glm('annualized_pay ~ male01 + hours + I(hours**2) + job_level + C(department) + C(job_family)',
                    data=glm_df, family=Gamma(link=Log())).fit(cov_type='HC1', maxiter=200)
    b = glmr.params.get('male01', np.nan)
    ci = glmr.conf_int().loc['male01'].tolist() if 'male01' in glmr.params.index else [np.nan, np.nan]
    e, lo, hi = (np.exp(b)-1)*100, (np.exp(ci[0])-1)*100, (np.exp(ci[1])-1)*100
    print('GLM (Gamma log-link, HC1): male effect = {:.2f}% (95% CI {:.2f}% to {:.2f}%)'.format(e, lo, hi))
    # Cluster-robust by department
    dept_codes = glm_df['department'].astype('category').cat.codes.values
    glmr_cl = glmr.get_robustcov_results(cov_type='cluster', groups=dept_codes)
    b = glmr_cl.params.item(glmr_cl.params.index.get_loc('male01')) if hasattr(glmr_cl.params,'index') else glmr_cl.params[list(glmr_cl.params.index).index('male01')]
    ci = glmr_cl.conf_int().loc['male01']
    e, lo, hi = (np.exp(b)-1)*100, (np.exp(ci[0])-1)*100, (np.exp(ci[1])-1)*100
    print('GLM (Gamma log-link, cluster by dept): male effect = {:.2f}% (95% CI {:.2f}% to {:.2f}%)'.format(e, lo, hi))
except Exception as e:
    try:
        # Fallback to ridge-regularized GLM to resolve singularities; standardize numeric terms for stability
        glm_df = glm_df.copy()
        for col in ['hours','job_level']:
            x = pd.to_numeric(glm_df[col], errors='coerce')
            glm_df[col+'_z'] = (x - x.mean())/x.std(ddof=0) if x.std(ddof=0) > 0 else x
        glmr = smf.glm('annualized_pay ~ male01 + hours_z + I(hours_z**2) + job_level_z + C(department) + C(job_family)',
                       data=glm_df, family=Gamma(link=Log())).fit_regularized(alpha=1e-3, L1_wt=0.0, maxiter=5000, cnvrg_tol=1e-4)
        b = glmr.params.get('male01', np.nan) if hasattr(glmr, 'params') else np.nan
        e = (np.exp(b)-1)*100 if np.isfinite(b) else np.nan
        print('GLM (Gamma log-link, ridge-regularized): male effect = {:.2f}%'.format(e))
    except Exception as e2:
        print('GLM failed:', e2)


In [ ]:
# Doubly Robust ATE (AIPW / DML) with cross-fitting
import numpy as np, pandas as pd, json, os
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import clone

# Optional: accelerate nuisance models with PyTorch MPS (Apple GPU) if available
import os as _dml_os
USE_TORCH = bool(globals().get('HAS_MPS', False) and _dml_os.environ.get('USE_MPS_DML','0') == '1')
if USE_TORCH:
    try:
        import torch
        import torch.nn as nn
        import torch.optim as optim
        DEVICE = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
    except Exception:
        USE_TORCH = False

base = gpd[['log_annual','male01','department','job_family','job_level','hours','is_salary']].dropna().copy()
y = base['log_annual'].values
t = base['male01'].astype(int).values
X = base[['department','job_family','job_level','hours','is_salary']].copy()
pre = ColumnTransformer([('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), ['department','job_family']),
                           ('num', 'passthrough', ['job_level','hours','is_salary'])])

skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
psi = np.zeros(len(base))
p_all = np.zeros(len(base))
fold_idx = np.zeros(len(base), dtype=int)
for k, (tr, te) in enumerate(skf.split(X, t)):
    if USE_TORCH:
        # Fit preprocessor on training fold only
        pre_k = clone(pre)
        Xtr_enc = pre_k.fit_transform(X.iloc[tr])
        Xte_enc = pre_k.transform(X.iloc[te])
        Xtr_enc = np.asarray(Xtr_enc, dtype=np.float32)
        Xte_enc = np.asarray(Xte_enc, dtype=np.float32)

        # Simple MLP architectures
        class MLP(nn.Module):
            def __init__(self, in_dim, out_dim=1):
                super().__init__()
                self.net = nn.Sequential(
                    nn.Linear(in_dim, 128), nn.ReLU(),
                    nn.Linear(128, 64), nn.ReLU(),
                    nn.Linear(64, out_dim)
                )
            def forward(self, x):
                return self.net(x)

        def fit_mlp_reg(Xn, yn, epochs=50):
            model = MLP(Xn.shape[1], 1).to(DEVICE)
            opt = optim.Adam(model.parameters(), lr=1e-3)
            lossf = nn.MSELoss()
            xb = torch.from_numpy(Xn).to(DEVICE)
            yb = torch.from_numpy(yn.astype(np.float32)).view(-1,1).to(DEVICE)
            model.train()
            for _ in range(epochs):
                opt.zero_grad(); pred = model(xb); loss = lossf(pred, yb); loss.backward(); opt.step()
            model.eval()
            return model

        def fit_mlp_clf(Xn, tn, epochs=50):
            model = MLP(Xn.shape[1], 1).to(DEVICE)
            opt = optim.Adam(model.parameters(), lr=1e-3)
            lossf = nn.BCEWithLogitsLoss()
            xb = torch.from_numpy(Xn).to(DEVICE)
            yb = torch.from_numpy(tn.astype(np.float32)).view(-1,1).to(DEVICE)
            model.train()
            for _ in range(epochs):
                opt.zero_grad(); logits = model(xb); loss = lossf(logits, yb); loss.backward(); opt.step()
            model.eval()
            return model

        # Propensity
        clf = fit_mlp_clf(Xtr_enc, t[tr].astype(np.float32))
        with torch.no_grad():
            p_hat = torch.sigmoid(clf(torch.from_numpy(Xte_enc).to(DEVICE))).cpu().numpy().ravel()
        p_hat = np.clip(p_hat, 0.01, 0.99)
        # Outcome models
        rf1 = fit_mlp_reg(Xtr_enc[t[tr]==1], y[tr][t[tr]==1].astype(np.float32))
        rf0 = fit_mlp_reg(Xtr_enc[t[tr]==0], y[tr][t[tr]==0].astype(np.float32))
        with torch.no_grad():
            m1 = rf1(torch.from_numpy(Xte_enc).to(DEVICE)).cpu().numpy().ravel()
            m0 = rf0(torch.from_numpy(Xte_enc).to(DEVICE)).cpu().numpy().ravel()
    else:
        # Propensity model (logistic)
        lr = Pipeline(steps=[('pre', clone(pre)), ('clf', LogisticRegression(max_iter=5000, solver='lbfgs'))])
        lr.fit(X.iloc[tr], t[tr])
        p_hat = lr.predict_proba(X.iloc[te])[:,1]
        p_hat = np.clip(p_hat, 0.01, 0.99)
        # Outcome models (T-learner with RF)
        rf1 = Pipeline(steps=[('pre', clone(pre)), ('rf', RandomForestRegressor(n_estimators=400, random_state=42, n_jobs=-1))])
        rf0 = Pipeline(steps=[('pre', clone(pre)), ('rf', RandomForestRegressor(n_estimators=400, random_state=42, n_jobs=-1))])
        rf1.fit(X.iloc[tr][t[tr]==1], y[tr][t[tr]==1])
        rf0.fit(X.iloc[tr][t[tr]==0], y[tr][t[tr]==0])
        m1 = rf1.predict(X.iloc[te])
        m0 = rf0.predict(X.iloc[te])
    psi_te = m1 - m0 + t[te]*(y[te] - m1)/p_hat - (1 - t[te])*(y[te] - m0)/(1 - p_hat)
    psi[te] = psi_te
    p_all[te] = p_hat
    fold_idx[te] = k
ate = float(np.mean(psi))
se = float(np.std(psi, ddof=1)/np.sqrt(len(psi)))
ci = (ate - 1.96*se, ate + 1.96*se)
ate_pct = (np.exp(ate)-1)*100.0
ci_pct = ((np.exp(ci[0])-1)*100.0, (np.exp(ci[1])-1)*100.0)
print('DML ATE on log(pay): {:.6f} (95% CI {:.6f} to {:.6f})'.format(ate, ci[0], ci[1]))
print('DML ATE in % terms: {:.2f}% (95% CI {:.2f}% to {:.2f}%)'.format(ate_pct, ci_pct[0], ci_pct[1]))
# Trimmed ATE (PS in [0.05,0.95])
mask = (p_all >= 0.05) & (p_all <= 0.95)
psi_trim = psi[mask]
ate_t = float(np.mean(psi_trim))
se_t = float(np.std(psi_trim, ddof=1)/np.sqrt(len(psi_trim)))
ci_t = (ate_t - 1.96*se_t, ate_t + 1.96*se_t)
ate_t_pct = (np.exp(ate_t)-1)*100.0
ci_t_pct = ((np.exp(ci_t[0])-1)*100.0, (np.exp(ci_t[1])-1)*100.0)
print('DML (trimmed) ATE in % terms: {:.2f}% (95% CI {:.2f}% to {:.2f}%)'.format(ate_t_pct, ci_t_pct[0], ci_t_pct[1]))
# Save summary
os.makedirs('artifacts', exist_ok=True)
open('artifacts/dml_ate_summary.json','w').write(json.dumps({
    'ate_log': ate, 'ci_log': ci, 'ate_pct': ate_pct, 'ci_pct': ci_pct,
    'ate_pct_trim': ate_t_pct, 'ci_pct_trim': ci_t_pct
}, indent=2))


In [ ]:

# Robust Linear Model (HuberT) with FE; robust to heavy tails and outliers
import statsmodels.api as sm, statsmodels.formula.api as smf

try:
    # Use explicit polynomial terms for hours to avoid reliance on unavailable poly()
    rlm_res = smf.rlm('log_annual ~ male01 + hours + I(hours**2) + I(hours**3) + job_level + C(department) + C(job_family)', data=gpd, M=sm.robust.norms.HuberT()).fit()
    b = rlm_res.params.get('male01', float('nan'))
    ci = rlm_res.conf_int().loc['male01'] if 'male01' in rlm_res.params.index else [float('nan'), float('nan')]
    print('RLM (HuberT): male effect = {:.2f}% (95% CI {:.2f}% to {:.2f}%)'.format((np.exp(b)-1)*100, (np.exp(ci[0])-1)*100, (np.exp(ci[1])-1)*100))
except Exception as e:
    print('RLM failed:', e)


In [ ]:
# Save consolidated results summary to JSON for auto-markdown
import json, math, os
os.makedirs('artifacts', exist_ok=True)

def pct_from_log(b):
    try:
        return (math.exp(float(b)) - 1.0) * 100.0
    except Exception:
        return float('nan')

def pack(model, name='male01'):
    try:
        b = float(model.params.get(name, float('nan')))
        ci = model.conf_int().loc[name].tolist() if name in model.params.index else [float('nan'), float('nan')]
        return {'coef_log': b, 'effect_pct': pct_from_log(b), 'ci_pct': [pct_from_log(ci[0]), pct_from_log(ci[1])]} 
    except Exception:
        return None

hours_basis = globals().get('used', globals().get('hours_basis', None))
fe_deg = globals().get('deg_used', None)
fe_ols = globals().get('m3', globals().get('m3_ols', None))
rlm = globals().get('rlm_res', None)
glm = globals().get('glmr', None)
ate_wls = globals().get('wres', None)
ate_wls_trim = globals().get('wres_trim', None)

res_json = {
    'data': 'data/chicago_salaries.csv',
    'hours_basis': str(hours_basis) if hours_basis is not None else None,
    'fe_deg': int(fe_deg) if isinstance(fe_deg, (int, float)) else None,
    'shap': globals().get('SHAP_INFO', None),
    'models': {
        'fe_ols': pack(fe_ols),
        'rlm': pack(rlm),
        'glm': pack(glm),
        'ate_wls': pack(ate_wls),
        'ate_wls_trim': pack(ate_wls_trim),
    }
}
with open('artifacts/results_summary.json','w') as fh:
    json.dump(res_json, fh, indent=2)
print('Wrote artifacts/results_summary.json')


In [ ]:
# Render auto-updating markdown summary from results_summary.json
import json, math
try:
    from IPython.display import Markdown, display
except Exception:
    Markdown = None
    def display(x):
        pass

def fmt_ci(lo, hi):
    try:
        return f"{float(lo):.2f}% to {float(hi):.2f}%"
    except Exception:
        return 'n/a'

with open('artifacts/results_summary.json') as fh:
    R = json.load(fh)

md = []
md.append('# Chicago Salaries: Final Summary (Auto-Generated)')
md.append('')
md.append(f"- Data: `{R.get('data','n/a')}`  ")
md.append(f"- Hours basis: `{R.get('hours_basis','n/a')}`  ")
md.append(f"- FE hours degree: `{R.get('fe_deg','n/a')}`  ")
sh = R.get('shap')
if sh:
    md.append(f"- SHAP model: `{sh.get('model','n/a')}` on {sh.get('n_eval','?')} rows  ")
md.append('')

M = R.get('models', {})
for label, key in [('Within-role FE OLS (HC1)','fe_ols'),
                   ('RLM (HuberT)','rlm'),
                   ('GLM (Gamma log link)','glm'),
                   ('ATE weighted FE','ate_wls'),
                   ('ATE trimmed FE','ate_wls_trim')]:
    v = M.get(key)
    if v:
        md.append(f"**{label}**  ")
        md.append(f"- Male effect (%): {v.get('effect_pct','n/a'):.2f}%  ")
        ci = v.get('ci_pct', [None,None])
        md.append(f"- 95% CI: {fmt_ci(ci[0], ci[1])}")
        md.append('')

md.append('**Interpretation**')
md.append('- Within-role FE OLS is the primary like-for-like estimate. RLM and GLM corroborate under heavy tails/heteroskedasticity.')
md.append('- The ATE complements FE OLS by balancing men and women on observables across the workforce; trimmed ATE reports an overlap-safe effect.')
md.append('- Hours effects use within-group centering + orthogonal basis to control multicollinearity; VIF and rank checks enforce a stable design.')

# Expanded, reader-friendly narrative
md.append('')
md.append('## What The Results Mean')
md.append('- Raw pay distributions show a long right tail (a few very high earners). The log scale makes the overall shape easier to compare: see `artifacts/pay_distribution_hist.png`.')
md.append('- Men and women are not spread evenly across departments and job families (e.g., more men in POLICE and FIRE). This “who works where” pattern matters for pay: see `artifacts/top_categories_bars.png`.')
md.append('- “Like-for-like” regression with department and job-family fixed effects (plus job level and hours terms) estimates the male effect on log pay near zero (well under 1%). With HC1 and department-clustered SEs, the effect is at most a few tenths of a percent — small in magnitude and borderline in significance.')
md.append('- Across the whole workforce (not holding roles constant), ATE weighting indicates men earn about 7–8% more on average. That gap reflects differences in role/department mix rather than pay within the same role.')
md.append('- Robust (Huber) and quantile models agree that outliers and tails do not drive the within-role finding; median effects are near zero.')
md.append('- Model diagnostics flag heteroskedasticity and some nonlinearity, so we rely on robust standard errors and flexible hours terms. Independence/statistical rank conditions are satisfied after de-duplication of collinear terms.')

md.append('## Visuals — How to Read Them')
md.append('- Pay Histograms: Raw vs log-scale. Log-scale makes typical pay differences more interpretable because it compresses very high values (`artifacts/pay_distribution_hist.png`).')
md.append('- Top Departments/Job Families: Bars show where people work the most; composition differences explain much of the raw gap (`artifacts/top_categories_bars.png`).')
md.append('- SHAP Summary: Global feature importance for predicting log pay. Hours, job level, and department/family dummies dominate; gender has low global importance (`artifacts/shap_summary_beeswarm.png`, `artifacts/shap_importance_bar.png`).')
md.append('- Influence & Residual Checks: Outlier and assumption diagnostics confirm robust inferences are appropriate (`artifacts/influence_scatter_clean.png`).')

md.append('## Bottom Line (Plain English)')
md.append('- There is a large raw difference in average pay between men and women in the City of Chicago workforce. But once we compare men and women doing the same type of work in the same departments (controlling for job family, department, job level, and hours), the remaining gender difference in pay is extremely small — under 1% — and not economically meaningful.')
md.append('- The across-workforce gap (~7–8%) is best explained by who works where and in which roles. In short, the data support the conclusion that role and department mix drive the observed average gap; within the same role and department, pay is effectively the same by gender.')
md.append('- Caveat: Gender is inferred from first names (some ambiguity), and we analyze one dataset. Even so, multiple robust models point to the same conclusion.')

content = "\n".join(md)
try:
    if Markdown is not None:
        display(Markdown(content))
    else:
        raise ImportError('IPython not available')
except Exception:
    print(content)
    import os
    os.makedirs('artifacts', exist_ok=True)
    open('artifacts/summary.md','w').write(content)


## Final Narrative Summary

### What The Results Mean
- Raw pay distributions are skewed with a long right tail; plotting log pay makes typical values clearer (`artifacts/pay_distribution_hist.png`).
- Men and women are not spread evenly across departments and job families (e.g., more men in POLICE and FIRE). This composition matters for average pay differences (`artifacts/top_categories_bars.png`).
- Like-for-like comparisons (same department and job family, controlling for job level and hours) show a remaining gender effect on log pay that is very small (well under 1%) and not economically meaningful. Robust and cluster-adjusted inferences agree.
- Across the workforce (without holding roles constant), the average pay gap is larger (~7–8%) and is explained by where people work and which roles they hold, not different pay for the same role.
- Robust (Huber) and quantile models indicate that outliers and tails do not drive the within-role finding; median effects are also near zero.
- Diagnostics flag unequal variance and some nonlinearity, so we use robust standard errors and flexible hours terms. Independence and rank conditions are satisfied after reducing collinearity.

### Visuals — How to Read Them
- Pay Histograms: Raw vs log-scale; the log scale compresses extreme values and makes typical differences clearer (`artifacts/pay_distribution_hist.png`).
- Top Departments/Job Families: Bars show where people work most; composition differences explain much of the raw gap (`artifacts/top_categories_bars.png`).
- SHAP Summary: Global feature importance for predicting log pay; hours, job level, and department/family dominate; gender has low global importance (`artifacts/shap_summary_beeswarm.png`, `artifacts/shap_importance_bar.png`).
- Influence & Residual Checks: Outlier/assumption diagnostics support using robust inferences (`artifacts/influence_scatter_clean.png`).

### Bottom Line (Plain English)
- There is a sizeable raw difference in average pay between men and women in the City workforce. But when we compare men and women doing the same type of work in the same departments, the remaining gender difference is effectively zero.
- The across-workforce gap (~7–8%) is best explained by who works where and in which roles. In short, role and department mix drive the observed average gap; within the same role and department, pay is essentially the same by gender.
- Caveat: Gender is inferred from first names (some ambiguity), and we analyze one dataset. Even so, multiple robust models point to the same conclusion.


## Key Visuals (with brief explanations)

**Pay histograms (raw and log)**
![Pay distribution](artifacts/pay_distribution_hist.png)
Raw scale shows a few very high earners; log scale makes typical pay clearer.

**Top departments and job families (counts)**
![Top categories](artifacts/top_categories_bars.png)
Men and women are distributed differently across roles/departments; composition drives the raw gap.

**SHAP summary (beeswarm)**
![SHAP beeswarm](artifacts/shap_summary_beeswarm.png)
Hours, job level, and department/family features dominate predicted pay; gender has low global importance.

**SHAP importance (bar)**
![SHAP importance](artifacts/shap_importance_bar.png)
Mean absolute SHAP values confirm which features matter most overall.

**Influence and residual diagnostics**
![Influence diagnostics](artifacts/influence_scatter_clean.png)
Highlights high-leverage/outlying points; supports using robust standard errors.


## Glossary of Methods and Acronyms
- FE (Fixed Effects): Controls for group differences (e.g., department, job family) so comparisons are like-for-like within groups.
- HC1 robust SEs: Standard errors adjusted for unequal variance across observations.
- Cluster-robust SEs: Standard errors that allow correlation within clusters (here, departments).
- ATE (Average Treatment Effect): Average difference in outcome if everyone had the ‘treatment’ (male) vs ‘control’ (female), balancing observables.
- Propensity score: Estimated probability of being in the treatment group given observed features; used to balance groups.
- DML (Double/Debiased Machine Learning): Uses flexible models for nuisance parts (propensity/outcome) while keeping the final effect unbiased.
- GLM (Gamma, log link): Model for positive, skewed outcomes; relates log(expected pay) to predictors.
- RLM (Robust Linear Model, Huber): Regression that downweights outliers to protect estimates.
- Quantile regression: Models medians (or other quantiles) instead of means; robust to skew/outliers.
- SHAP (SHapley Additive exPlanations): Consistent feature importance values that explain model predictions.
- VIF (Variance Inflation Factor): Indicator of multicollinearity; very high VIF suggests overlapping information between predictors.
- RESET test: Checks if a linear model is missing nonlinear terms or interactions.
- Durbin–Watson (DW): Tests residual autocorrelation; ≈2 suggests independence.
- Breusch–Pagan (BP) / White tests: Tests for unequal variance (heteroskedasticity).
- Oaxaca–Blinder: Decomposes the raw gap into explained (composition) and unexplained parts.
- Orthogonal polynomials (Legendre): Curved terms for hours that avoid collinearity with other predictors.
- 95% CI: Range likely containing the true effect; if it includes 0%, the effect is consistent with no difference.
- Log pay: Using log(pay) turns percentage effects into additive terms and stabilizes variance.
- MPS (Apple GPU): Apple’s GPU acceleration used for some ML steps (optional).
